In [ ]:
#install.packages(c("tidyverse", "reshape2"))
# install.packages("fmsb")
library(tidyverse) # loads dplyr, ggplot2, stringr etc
library(reshape2) # for melt an cast

#install.packages(c("car","agricolae", "FSA"))
library(car)
library(agricolae)
library(FSA)

In [ ]:
df <- read.csv("3_phytohormones.csv",
                          header = TRUE)

In [ ]:
# Fold change
control <- df %>% filter(Treatment == "Control") %>% select(Soil, 
                                                            Sample_Type, 
                                                            Phytohormone, 
                                                            Concentration_ng_g)

control <- control %>% group_by(Soil, Sample_Type, Phytohormone) %>% summarise(mean_ng = mean(Concentration_ng_g, na.rm = TRUE))
names(control)[4] <- "mean_ng"

foldchange <- df %>% filter(Treatment == "Insect")
foldchange <- left_join(foldchange, control, relationship = "many-to-many", by = join_by(Soil, Sample_Type, Phytohormone))
foldchange <- foldchange %>% group_by(Soil, Sample_Type, Phytohormone) %>% mutate(fold_change = Concentration_ng_g / mean_ng)

In [ ]:
##STATS##–––––––– 

# CHECKING NORMAL DISTRIBUTION

# FOR REGULAR DATA -- Check assumptions: normality and homogeneity of variances

shapiro = as.numeric(shapiro.test(df$Concentration_ng_g)$p.value)
levene = as.numeric(car::leveneTest(Concentration_ng_g ~ Soil, data=df)$`Pr(>F)`[1])
if (shapiro > 0.05 & levene > 0.05) {
  print("     you should perform ANOVA test")
} else {
  print("    you should perform Kruskal-Wallis test")
}
print(paste0("Shapiro test p-value: ", shapiro))
print(paste0("Levene test p-value: ", levene))

# FOR LOG-FOLD DATA -- Check assumptions: normality and homogeneity of variances

shapiro = as.numeric(shapiro.test(foldchange$fold_change)$p.value)
levene = as.numeric(car::leveneTest(fold_change ~ Soil, data=foldchange)$`Pr(>F)`[1])
if (shapiro > 0.05 & levene > 0.05) {
  print("you should perform ANOVA test")
} else {
  print("you should perform Kruskal-Wallis test")
}
print(paste0("Shapiro test p-value: ", shapiro))
print(paste0("Levene test p-value: ", levene))


In [ ]:
# Preparation of dataframe containing statistical groups

annotations <- data.frame(a = rep(NA, times = 16),
                          b = rep(NA, times = 16),
                          c = rep(NA, times = 16), 
                          d = rep(NA, times = 16), 
                          e = rep(NA, times = 16), 
                          a.name = rep(NA, times = 16),
                          b.name = rep(NA, times = 16),
                          c.name = rep(NA, times = 16),
                          d.name = rep(NA, times = 16),
                          e.name = rep(NA, times = 16)
                        )

# Sub df's of the sample and treatment groups

df$condition <- paste0(df$Soil, "_", df$Treatment)
leaf <- df %>% filter(Sample_Type == "Leaf")
root <- df %>% filter(Sample_Type == "Root")


phytohormones <- unique(df$Phytohormone) 
# Calculate K-W p-values to determine groups and place those groups into empty dataframe called annotations with corresponding condition. 
# (The agricolae function outputs the results in different condition orders, so it's 
# important to ensure that the group letter is listed with the corresponding condition).

stats1 <- c() # Adding a vector to collect statistics to paste on graph

for (i in phytohormones[1:5]) {
    grp <- c() # empty vector to collect the statistical groups
    name <- c() # empty vector to collect corresponding condition name

    pos.grp <- which(phytohormones == i) # where to place group and name information
    pos.name <- which(phytohormones == i) + 5

    filter.leaf <- leaf %>% filter(Phytohormone == i) # filter for the BXD of interest in each sub df
    filter.root <- root %>% filter(Phytohormone == i)

    kw.leaf <- agricolae::kruskal(filter.leaf$Concentration_ng_g, # Kruskal-Wallis for leaf samples of control treatment
                                filter.leaf$condition, alpha = 0.05, p.adj = "fdr")
    kwres.leaf <- rownames_to_column(kw.leaf$groups, var="Name")
    grp <- c(grp, kwres.leaf$groups)
    name <- c(name, kwres.leaf$Name)

    kw.root <- agricolae::kruskal(filter.root$Concentration_ng_g, # Kruskal-Wallis for leaf samples of insect treatment
                                filter.root$condition, alpha = 0.05, p.adj = "fdr")
    kwres.root <- rownames_to_column(kw.root$groups, var="Name")
    grp <- c(grp, kwres.root$groups)
    name <- c(name, kwres.root$Name)

    stats1 <- c(stats1, i, kw.leaf$statistics, kw.root$statistics)
    
    annotations[,pos.grp] <- grp
    annotations[,pos.name] <- name

    }

# Position of annotations
positions <- data.frame(a.pos = rep(c(3750, 500), each = 8), 
                       b.pos = rep(c(400, 130), each = 8),
                       c.pos = rep(c(100, 45), each = 8), 
                       d.pos = rep(c(100, 9), each = 8), 
                       e.pos = rep(c(875, 105), each = 8))

annotations <- cbind(annotations, positions)


annotations$Sample_Type <- rep(c("Leaf", "Root"), each = 8)

sample.labs <- c("Leaf samples", "Root samples")
names(sample.labs) <- c("Leaf", "Root")
conditions <- unique(df$condition)

# Graphing 

for (i in phytohormones[1:5]) { # there are 5 phytohormones
    
    og.name <- names(annotations)[which(phytohormones == i) + 5] 
    names(annotations)[which(phytohormones == i) + 5] <- "condition" 

    pos <- which(phytohormones == i)+10
    lab <- which(phytohormones == i)

    filtered <- df %>% filter(Phytohormone == i) # filter for the BXD of interest
    
    plot <- ggplot(filtered, aes(x=condition, y=Concentration_ng_g)) + 
            geom_boxplot(outlier.alpha = 0.9, outlier.shape = 16, aes(fill=Soil)) +
            geom_jitter(alpha=0.5, size=1, aes(shape = Treatment), show.legend = TRUE) + 
            scale_shape_manual(values = c(16,1)) +
            geom_text(data=annotations, mapping = aes(x = condition, y = annotations[,pos], label = annotations[,lab])) +
            facet_grid(Sample_Type~., scale = "free",  labeller = labeller(Sample_Type = sample.labs)) +
            geom_vline(xintercept=c(2.5,4.5,6.5), linetype = "dashed", color = "grey") +
            labs(y = paste0("µg/g ", i)) + 
            scale_fill_manual(values = c("#877cba", "#a35da5", "#69b19a", "#8ec372")) +
            theme_classic() + 
            theme(strip.background = element_rect(fill=NA, color = NA), strip.text = element_text(colour = "black")) +
            theme(axis.line = element_line(color='black'),
                plot.background = element_blank(),
                panel.grid.minor = element_blank(),
                panel.grid.major = element_blank()) +
            theme(strip.text = element_text(size = 12)) + 
            theme(axis.text.x = element_text(angle=45, vjust=1, hjust=1))
    
    print(plot)
    #ggsave(paste("~/Desktop/", i, ".pdf", sep = ""))
    names(annotations)[which(phytohormones == i) + 5] <- og.name # restoring original column name
    }


In [ ]:
# Preparation of dataframe containing statistical groups

annotations2 <- data.frame(a = rep(NA, times = 8),
                           b = rep(NA, times = 8),
                           c = rep(NA, times = 8),
                           d = rep(NA, times = 8),
                           e = rep(NA, times = 8),
                           a.name = rep(NA, times = 8),
                           b.name = rep(NA, times = 8),
                           c.name = rep(NA, times = 8),
                           d.name = rep(NA, times = 8),
                           e.name = rep(NA, times = 8))

#new.foldchange <- new.foldchange %>% mutate(fold_change = ifelse(is.na(fold_change), 0, fold_change))

# Sub df's of the sample and treatment groups
leaf2 <- foldchange %>% filter(Sample_Type == "Leaf")
root2 <- foldchange %>% filter(Sample_Type == "Root")

stats2 <- c()

for (i in phytohormones) {
    grp <- c()
    name <- c()
    pos.grp <- which(phytohormones == i)
    pos.name <- which(phytohormones == i) + 5

    filter.leaf <- leaf2 %>% filter(Phytohormone == i)
    filter.root <- root2 %>% filter(Phytohormone == i)
    
    kw.leaf <- agricolae::kruskal(log2(filter.leaf$fold_change), # Kruskal-Wallis for leaf samples of control treatment
                                  filter.leaf$Soil, alpha = 0.05, p.adj = "fdr")
    kwres.l <- rownames_to_column(kw.leaf$groups, var="Name")
    grp <- c(grp, kwres.l$groups)
    name <- c(name, kwres.l$Name)


    kw.root <- agricolae::kruskal(log2(filter.root$fold_change), # Kruskal-Wallis for root samples of control treatment
                        filter.root$Soil, alpha = 0.05, p.adj = "fdr")
    kwres.r <- rownames_to_column(kw.root$groups, var="Name")
    grp <- c(grp, kwres.r$groups)
    name <- c(name, kwres.r$Name)

    stats2 <- c(stats2, i, kw.leaf$statistics, kw.root$statistics) # collect chisq and p.values to put on graphs
    
    annotations2[,pos.grp] <- grp
    annotations2[,pos.name] <- name

}

annotations2$Sample_Type <- rep(c("Leaf", "Root"), each = 4)

sample.labs <- c("Leaf samples", "Root samples")
names(sample.labs) <- c("Leaf", "Root")

# Graphing 

for (i in phytohormones) { 

    og.name <- names(annotations2)[which(phytohormones == i) + 5] 
    names(annotations2)[which(phytohormones == i) + 5] <- "Soil" 

    lab <- which(phytohormones == i)

    filtered <- foldchange %>% filter(Phytohormone == i) # filter for the BXD of interest

    plot <- ggplot(filtered, aes(x=Soil, y=log2(fold_change), fill = Soil)) + 
            geom_boxplot(outlier.alpha = 0.65, outlier.shape = 17) +
            geom_jitter(alpha=0.5, size=1, shape=17, show.legend = FALSE) + 
            facet_grid(.~Sample_Type, scale = "free",  labeller = labeller(Sample = sample.labs)) +
            geom_text(data=annotations2, mapping = aes(x = Soil, y = 8.5, label = annotations2[,lab])) +
            labs(y = paste0("log2 fold change ", i), x = "Soil") + 
            scale_fill_manual(values = c("#877cba", "#a35da5", "#69b19a", "#8ec372")) +
            scale_y_continuous(limits = c(-7,8.5)) +
            geom_hline(yintercept=0, linetype = "dotted") +
            theme_linedraw() + 
            theme(strip.background = element_rect(fill=NA, color = NA), strip.text = element_text(colour = "black")) +
            theme(axis.line = element_line(color='black'),
                plot.background = element_blank(),
                panel.grid.minor = element_blank(),
                panel.grid.major = element_blank()) +
            theme(strip.text = element_text(size = 12))
    
    print(plot)
   #ggsave(paste("~/Desktop/", i, "_foldchange",".pdf", sep = ""))
    names(annotations2)[which(phytohormones == i) + 5] <- og.name # restoring original column name
    }


In [ ]:
#Spider plots
library(fmsb)

spider <- as.data.frame(df %>% group_by(Phytohormone, Sample_Type, Treatment, Soil)
                        %>% summarize(mean = mean(Concentration_ng_g, na.rm = TRUE)))
treatment <- unique(spider$Treatment)
soil <- unique(spider$Soil)
color <- c("#877cba", "#a35da5", "#69b19a", "#8ec372")

# Leaf spider plots
for (i in soil) {
    col <- color[which(soil == i)]
    spider_df <- spider %>% filter(Sample_Type == "Leaf" & Soil == i) %>% select(Phytohormone, Treatment, mean)
    spider_df <- dcast(spider_df, Treatment~Phytohormone) %>% select(-Treatment)
    rownames(spider_df) <- treatment
    spider_df <- rbind(c(100, 16, 28, 160, 2040), rep(0,5), spider_df)

    pdf(file = paste0('~/Desktop/phyto_leaf_', i, '.pdf')) 
    radarchart(spider_df, axistype = 1,
          pcol = col,
          plwd = 2.5,
          cglcol = "#000000",
          cglty = 1)
    dev.off()
}

# Root spider plots
for (i in soil) {
    col <- color[which(soil == i)]
    spider_df <- spider %>% filter(Sample_Type == "Root" & Soil == i) %>% select(Phytohormone, Treatment, mean)
    spider_df <- dcast(spider_df, Treatment~Phytohormone) %>% select(-Treatment)
    rownames(spider_df) <- treatment
    spider_df <- rbind(c(100, 16, 28, 160, 2040), rep(0,5), spider_df)

    pdf(file = paste0('~/Desktop/phyto_root_', i, '.pdf')) 
    radarchart(spider_df, axistype = 1,
          pcol = col,
          plwd = 2.5,
          cglcol = "#000000",
          cglty = 1)
    dev.off()
}